# Retargeting Pascal0 for Intel 64-bit Architecture
Final project for COMPSCI 4TB3: Syntax-Based Tools and Compilers

By James Priebe, McMaster University




## Introduction
The topic of this report is a new compiler backend which generates NASM-syntax 64 bit assembly code. Included are a summary of the changes made to account for differences from the original MIPS architecture as well as the full code (CGx86.py and P0.py; scanner and symbol table unchanged from original compiler) and test suite used.

### Changes from Proposal
This report was created using Jupyter, a live code documentation tool, rather than the literate programming tool noweb as originally planned.

## Calling Convention
The generated assembly code handles procedure calls similarly to MIPS. It adheres to the C 64-bit convention only for standard procedure calls printf and scanf (see section: Input and Output).

### Caller
The caller pushes arguments to the stack, and is responsible for cleaning the stack upon return. Every time an actual parameter is pushed, we increment the global variable stacksize, which is set back to zero after the call. Parameters are generated by the following function:

In [ ]:
def genActualPara(ap, fp, n):
    """Pass parameter, ap is actual parameter, fp is the formal parameter,
    either Ref or Var, n is the parameter number"""
    global stacksize

    if type(fp) == Ref:  #  reference parameter, assume p is Var

        if ap.adr != 0:  #  load address in register
            r = obtainReg()
            loadAddress(r, ap.reg, ap.adr)
        else: 
            r = ap.reg  #  address already in register
        
        putInstr('push ' + r)
        stacksize += 8 # we restore stack after return
        releaseReg(r)

    else:  #  value parameter
        if type(ap) != Cond:
            if type(ap) != Reg: 
                ap = loadItem(ap)
            putInstr('push ' + ap.reg)
            stacksize += 8
            releaseReg(ap.reg)
        else: mark('unsupported parameter type')

And the procedure call:

In [ ]:
def genCall(pr):
    """Assume pr is Proc"""
    global stacksize

    putInstr('call ' + pr.name)
    putInstr('add rsp, ' + str(stacksize)) #clean the stack
    stacksize = 0

Here is an example of generated code:

In [ ]:
main:

    ...

    mov r12, 7
    push r12
    lea r15, [x_]
    push r15
    call q
    add rsp, 16
    
    ...

### Callee
The callee is then responsible for creating a stack frame and making space for local variables. The stack has the following layout (RBP is the frame pointer register):

Arguments are stored in **reverse order** at RBP + 16, RBP + 24, ...
Local variables are store in **regular order** at RBP - 8, RBP - 16, ...

Here is an example demonstrating this:

In [ ]:
q:
    push rbp              
    mov rbp, rsp         #set up stack frame

    sub rsp, 32          #allocate space for 4 local variables
    mov r13, [40 + rbp]  #load the first argument (second, third, fourth at RBP + 32, RBP + 24, RBP +16)
    mov r9, [0 + r13]
    mov [-8 + rbp], r9   #store argument 1 in local variable 1

## Input and Output
x86 does not support printing integers directly using a system call. 
The compiler generates calls to the external c functions *printf* and *scanf*. We must adhere to the C 64 bit calling convention, which is different from 32 bit. The first 6 arguments must be stored in registers
RDI, RSI, RDX, RCX, R8, and R9. Additionally, printf and scanf require the number of floating-point arguments to be stored in RAX, which in our case is always zero.

Upon program entry the compiler generates the following boilerplate for reading integer and newlines and writing integers:

In [ ]:
    extern printf
    extern scanf
    global main
 
    section .data

newline:    db "", 10, 0
write_msg:  db "%d", 10, 0
read_msg:   db "Enter an integer: ", 0
read_format:    db "%d", 0


And the generator functions:

In [ ]:
def genWrite(x):

    putInstr('')
    putInstr('mov rdi, write_msg')
    loadItemReg(x, 'rsi')
    putInstr('mov rax, 0')
    putInstr('call printf')
    putInstr('')


def genWriteln():

    putInstr('')
    putInstr('mov rdi, newline')
    putInstr('mov rax, 0')
    putInstr('call printf')
    putInstr('')

def genRead(x):
    """Assume x is Var"""

    putInstr('')
    putInstr('mov rdi, read_msg')
    putInstr('mov rax, 0')
    putInstr('call printf')
    putInstr('mov rdi, read_format')
    putInstr('mov rsi, number')
    putInstr('mov rax, 0')
    putInstr('call scanf')
    putInstr('mov rsi, [number]')
    putInstr('mov [' + str(x.adr) + '], rsi')
    putInstr('')

## Register Usage
x_86 64 introduces eight new general-purpose registers, r8 - r15.
This greatly simplifies translation from MIPS, because the original "general purpose" registers, RDX, RSI, etc are not truly general purpose; for example they are used to store the results of division and comparison operations. Thus we use RCX and R8 - R15 as direct analogs of MIPS $t0 - $t8. Our compiler uses the other registers for the following special purposes:

**RBX**
Zero register. RBX is not nonvolatile. It can be set to zero at the beginning of main and will not be destroyed by procedure calls.

**RAX**
Number of floating point arguments for printf and scanf calls.
Also holds the quotient of div operation.

**RDX**
Holds remainder of div instruction. Used for modulo
Contains the format string for scanf and printf

**RSI**
Read/Write argument for printf/scanf

**RDI**
Format string for printf/scanf

## Addressing Mode
The compiler uses the following addressing modes, with example instructions:

**Register**
mov r8, r10
mul r8, r10

**Immediate**
mov rax, 0
add rsi, 8

**Direct/offset addressing**
lea r9, [x_ + 32]
mov [x_], r10 

For direct addressing the range is extended to -2^40 to 2^40 -1; the theoretical range of -2^64 to 2^64 -1 is not supported by current CPUs.

## Conditional Branching
Branching requires an additional operation from MIPS. The branch only takes one operation, the target address/label. First a *cmp* instruction must be generated, which sets the EFLAGS register that is used by the branch instruction. Below are the functions for generating conditional branches:

In [ ]:
def condOp(cd):
    """Assumes op in {EQ, NE, LT, LE, GT, GE}, return instruction mnemonic"""
    return 'je' if cd == EQ else \
           'jne' if cd == NE else \
           'jl' if cd == LT else \
           'jle' if cd == LE else \
           'jg' if cd == GT else \
           'jge'

def genCond(x):
    """Assume x is Bool, generate code for branching on x"""
    if type(x) != Cond: x = loadBool(x)
    neg = condOp(negate(x.cond))
    putInstr('cmp ' + x.left + ', ' + x.right)
    putInstr(neg + ' ' + x.labA[0])
    releaseReg(x.left); releaseReg(x.right); putLab(x.labB)
    return x

## Other Changes
#### Two-Operand Instructions
All instructions used by the compiler have at most two operands. Most arithmetic operations must be done in two steps, i.e, the x86 equivalent for the MIPS instruction

sub \$fp, \$sp, 4

is

mov rbp, rsp
sub rbp, 4

However many operations are of the form *x = x + y*, which can be done in a single instruction.
#### Division and modulus
Division and modulus are single-operand, register-only instructions in x86. The *idiv* instruction takes the divisor operand and stores the quotient in RAX and RDX, respectively. Below is the code generator for a modulus operation:

In [ ]:
def putModulo(x, y):
    if type(x) != Reg: x = loadItem(x)
    if x.reg == R0: x.reg, r = obtainReg(), R0
    else: r = x.reg # r is source, x.reg is destination
    if type(y) == Const:
        testRange(y) 
        putInstr('mov rax, ' + r)
        yc = obtainReg()
        putInstr('mov ' + yc + ', ' + str(y.val))
        putInstr('xor rdx, rdx')
        putInstr('idiv ' + yc)
        releaseReg(yc)
        # remainder is stored in rdx
        putInstr('mov ' + r + ', rdx')

    else:
        if type(y) != Reg: y = loadItem(y)
        putInstr('mov ' + x.reg + ', ' + r)
        putInstr('mov rax, ' + x.reg)
        putInstr('xor rdx, rdx')
        putInstr('idiv ' + y.reg)
        putInstr('mov ' + x.reg + ', rdx')
        releaseReg(y.reg)
    return x

#### Other files
P0.py has been modified to set the size of types Array and Boolean to 8 instead of 4.

## Testing
Comprehensively testing a compiler for correctness is large and difficult undertaking. Compilers of well-known languages can be tested by compiling a suitable large open-source project and running its own test suite. The Pascal0 compiler was tested for completeness with several programs designed to test specific attributes of the language, as well as with a factorial program for more robust coverage. Actual output matches expected output for all test cases. See tests folder for test programs. 

## Running the Compiler
The NASM compiler is required to compile and link the generated assembly code into an executable binary. A script, compile.py, has been included which will generate the assembler code and compile a 64-bit binary using NASM. Execute the script from the same directory as the compiler files as follows:

In [ ]:
python3 compile.py path/to/myfile.p

# Produces
path/to/myfile.s  #generated code
path/to/myfile    #executable

## Appendix A: Compiler Script
#### compile.p

In [ ]:
# %load compile.py
import os, sys

from P0 import compileString

#python3 Compile.py tests/filename

def compile(srcfn):

    if srcfn.endswith('.p'):
        with open(srcfn, 'r') as f: src = f.read()
        dstfn = srcfn[:-2] + '.s'
        objfn = srcfn[:-2] + '.o'
        binfn = srcfn[:-2]
        compileString(src, dstfn)

        os.system('nasm -f elf64 '+ dstfn + ' && gcc -m64 -o ' + binfn + ' ' + objfn)
        os.system('rm ' + objfn)

    else: print("'.p' file extension expected")

if __name__ == "__main__":
  compile(sys.argv[1])
